In [63]:
import pandas as pd
import numpy as np
import matplotlib as plt
import scipy.stats as stats
import glob 
import re
from datetime import datetime
import chardet

path = './finance_data/'

In [64]:
all_files = glob.glob(path + 'moex-IMOEX-*-marg-optionsdesk.csv')

In [65]:
with open(path + "moex-IMOEX-120325-marg-optionsdesk.csv", "rb") as f:
    result = chardet.detect(f.read())
    print(result["encoding"])

windows-1251


In [ ]:
dfs = []

for filename in all_files:
    date_match = re.search(r'IMOEX-(\d{6})', filename)
    
    if date_match:
        date_str = date_match.group(1)
        
        #convertion date to format dd.mm.yyyy
        day = date_str[:2]
        month = date_str[2:4]
        year = '20' + date_str[4:6]
        
        formatted_date = f"{day}.{month}.{year}"
        
        df = pd.read_csv(filename, encoding='windows-1251')
        
        df['execution_date'] = formatted_date
        dfs.append(df)
        print(f"Processed {filename} with date {formatted_date}")


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xce in position 6: invalid continuation byte

In [ ]:
if dfs:
    combined_df = pd.concat(dfs, ignore_index=True)
    
    combined_df['execution_date_dt'] = pd.to_datetime(combined_df['execution_date'], format='%d.%m.%Y')
    
    combined_df = combined_df.sort_values('execution_date_dt')
    
    combined_df.to_csv('combined_moex_options.csv', encoding='utf-8', index=False)
    
    print(f"Combined {len(dfs)} files into 'combined_moex_options.csv'")
    print(f"Shape of combined dataframe: {combined_df.shape}")
    
    # to verify if needed
    # print(combined_df.head())
else:
    print("No matching files found.")

Combined 15 files into 'combined_moex_options.csv'
Shape of combined dataframe: (759, 46)


In [ ]:
# df_combined = pd.read_csv('combined_moex_options.csv', encoding='cp1251') #cyrilic encoding

df_combined = pd.read_csv('combined_moex_options.csv', encoding='utf-8-sig') # windows-1251 is original encoding

df_combined.head()

,"CALL: Объем торгов, руб","CALL: Объем торгов, контр",CALL: Открыт.позиций,"CALL: Последняя сделка, Значение","CALL: Последняя сделка, Дата","CALL: Последняя сделка, Изменение",CALL: ПОКУПКА,CALL: ПРОДАЖА,CALL: Расчетная цена,CALL: Теоретическая цена,...,PUT: пїЅпїЅпїЅпїЅпїЅпїЅпїЅпїЅпїЅ пїЅпїЅпїЅпїЅ,PUT: пїЅпїЅпїЅпїЅпїЅпїЅпїЅ,PUT: пїЅпїЅпїЅпїЅпїЅпїЅпїЅ.1,"PUT: пїЅпїЅпїЅпїЅпїЅпїЅпїЅпїЅпїЅ пїЅпїЅпїЅпїЅпїЅпїЅ, пїЅпїЅпїЅпїЅпїЅпїЅпїЅпїЅ","PUT: пїЅпїЅпїЅпїЅпїЅпїЅпїЅпїЅпїЅ пїЅпїЅпїЅпїЅпїЅпїЅ, пїЅпїЅпїЅпїЅ","PUT: пїЅпїЅпїЅпїЅпїЅпїЅпїЅпїЅпїЅ пїЅпїЅпїЅпїЅпїЅпїЅ, пїЅпїЅпїЅпїЅпїЅпїЅпїЅпїЅпїЅ",PUT: пїЅпїЅпїЅпїЅпїЅпїЅ. пїЅпїЅпїЅпїЅпїЅпїЅпїЅ,"PUT: пїЅпїЅпїЅпїЅпїЅ пїЅпїЅпїЅпїЅпїЅпїЅ, пїЅпїЅпїЅ","PUT: пїЅпїЅпїЅпїЅпїЅ пїЅпїЅпїЅпїЅпїЅпїЅ, пїЅпїЅпїЅпїЅпїЅ",execution_date_dt
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,369.08,330.42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-03-12
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.12,0.01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-03-12
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.71,0.28,0.01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-03-12
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.75,0.68,0.03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-03-12
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.74,1.74,0.12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-03-12


In [31]:
# Вычисление математического ожидания и стандартного отклонения цены
math_expectation = df['Цена'].mean()
std_deviation = df['Цена'].std()
print(f"\nМатематическое ожидание цены: {math_expectation:.2f}")
print(f"Стандартное отклонение цены (волатильность): {std_deviation:.2f}")

# Простейшая стратегия:
# Если цена ниже математического ожидания, считаем это сигналом для покупки ("Покупка").
# Если цена выше математического ожидания, сигнал для продажи ("Продажа").
def generate_signal(price):
    if price < math_expectation:
        return 'Покупка'
    elif price > math_expectation:
        return 'Продажа'
    else:
        return 'Нейтрально'
    
df['Сигнал'] = df['Цена'].apply(generate_signal)

# Расчет логнормального распределения для цены
# Для подгонки логнормального распределения закрепляем loc=0 (floc=0)
shape, loc, scale = stats.lognorm.fit(df['Цена'], floc=0)
x = np.linspace(df['Цена'].min(), df['Цена'].max(), 100)
pdf_fitted = stats.lognorm.pdf(x, shape, loc, scale)

# Построение графиков
plt.figure(figsize=(14, 6))

# График 1: Историческая цена с сигналами покупки и продажи
plt.subplot(1, 2, 1)
plt.plot(df['Дата'], df['Цена'], label='Цена', color='blue')
# Отображаем сигналы покупки и продажи
buy_signals = df[df['Сигнал'] == 'Покупка']
sell_signals = df[df['Сигнал'] == 'Продажа']
plt.scatter(buy_signals['Дата'], buy_signals['Цена'], marker='^', color='green', label='Покупка', s=100)
plt.scatter(sell_signals['Дата'], sell_signals['Цена'], marker='v', color='red', label='Продажа', s=100)
plt.xlabel('Дата')
plt.ylabel('Цена')
plt.title('Историческая цена и торговые сигналы')
plt.legend()

# График 2: Гистограмма распределения цены и подгонка логнормального распределения
plt.subplot(1, 2, 2)
plt.hist(df['Цена'], bins=20, density=True, alpha=0.6, label='Гистограмма цены', color='skyblue')
plt.plot(x, pdf_fitted, 'r-', lw=2, label='Логнормальное распределение')
plt.xlabel('Цена')
plt.ylabel('Плотность распределения')
plt.title('Распределение цены')
plt.legend()

plt.tight_layout()
plt.show()

# Вычисление скользящего стандартного отклонения (20-дневная волатильность) ---
# этот показатель можно интерпретировать как оценку будущей волатильности.
df['Волатильность_20'] = df['Цена'].rolling(window=20).std()

# График скользящей волатильности
plt.figure(figsize=(10, 4))
plt.plot(df['Дата'], df['Волатильность_20'], label='20-дневная волатильность', color='purple')
plt.xlabel('Дата')
plt.ylabel('Стандартное отклонение')
plt.title('Будущая волатильность (20-дневное скользящее стандартное отклонение)')
plt.legend()
plt.show()

KeyError: 'Цена'